# (CNN) Global Average Pooling

Most of the networks used the `Convolutional layers` as feature extractors and then fed into `fully connected layers`, followed by an output layer. **Global average pooling means that you average each feature map separately.**


* PyTorch provides a slightly more versatile module called `nn.AdaptiveAvgPool2d()`, which averages a grid of activations into whatever sized destination you require.


In [109]:
import torch
import torch.nn as nn

def count_parameters(model):
    print('Total Parameters:')
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def fed_image_to_model(myModel, batch_size=1, width=28, height=28):
    print(myModel)
    print(f'Fed an image ({batch_size}, {1}, {height}, {width}) to Model')
    x = torch.rand([batch_size, 1, height, width])  
    output = myModel(x)

## 01 Fed into Fully Connected Layer

In [105]:
class CNNwithFullyConnected(nn.Module):
    def __init__(self):
        super(CNNwithFullyConnected, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        self.flatten = nn.Flatten()

        self.fullyconected = nn.Sequential(   
            nn.Linear(in_features=28*28*128, out_features=1024),
            nn.ReLU(),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        output = self.conv(x)
        print('After Conv Layer (without AdaptiveAvgPooliong):', output.shape)
        output = self.flatten(output)
        print('After Flatten:', output.shape)
        output = self.fullyconected(output)
        print('After Output Layer:', output.shape)
        return output

In [106]:
model = CNNwithFullyConnected()

In [110]:
fed_image_to_model(model)

CNNwithFullyConnected(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fullyconected): Sequential(
    (0): Linear(in_features=100352, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)
Fed an image (1, 1, 28, 28) to Model
After Conv Layer (without AdaptiveAvgPooliong): torch.Size([1, 128, 28, 28])
After Flatten: torch.Size([1, 100352])
After Output Layer: torch.Size([1, 10])


In [111]:
count_parameters(model) 

Total Parameters:


102846218

## 02 Using Adaptive Avg Pooling
<img src="https://peltarion.com/static/global_average_pooling_a.png" width="600" />

Image source [here](https://peltarion.com/knowledge-center/modeling-view/build-an-ai-model/blocks/global-average-pooling-2d).

In [118]:
class CNNwithGloabalAvGPooling(nn.Module):
    def __init__(self):
        super(CNNwithGloabalAvGPooling, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.avgpooling = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.linear_layer = nn.Sequential(
            nn.Linear(in_features=128, out_features=32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )
        
    def forward(self, x):
        output = self.conv(x)
        print('After Conv Layer:', output.shape)
        output = self.avgpooling(output)
        print('After AdaptiveAvgPooling:', output.shape)
        output = self.flatten(output)
        print('After Flatten:', output.shape)
        output = self.linear_layer(output)
        print('After Output Layer:', output.shape)
        return output

In [115]:
model2 = CNNwithGloabalAvGPooling()

In [117]:
fed_image_to_model(model2)

CNNwithGloabalAvGPooling(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (avgpooling): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layer): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
  )
)
Fed an image (1, 1, 28, 28) to Model
After Conv Layer: torch.Size([1, 128, 28, 28])
After AdaptiveAvgPool: torch.Size([1, 128, 1, 1])
After Flatten: torch.Size([1, 128])
After Output Layer: torch.Size([1, 10])


In [119]:
count_parameters(model2)

Total Parameters:


78954

## Examples

In [123]:
image = torch.rand(1, 128, 28, 28)

# target output size of 5x7
m = nn.AdaptiveAvgPool2d((5,7))
output = m(image)
print(output.shape)

# target output size of 7x7 (square)
m = nn.AdaptiveAvgPool2d(7)
output = m(image)
print(output.shape)

# target output size of 28x7
m = nn.AdaptiveAvgPool2d((None, 7))
output = m(image)
print(output.shape)

torch.Size([1, 128, 5, 7])
torch.Size([1, 128, 7, 7])
torch.Size([1, 128, 28, 7])
